# Evaluation component

## Setting environment

### Install Jupyter Kernel and python version

In [1]:
%%bash
chmod +x config/install_env_kernel.sh
bash config/install_env_kernel.sh -n 310-iris -p 3.10



Reading package lists...
Building dependency tree...
Reading state information...
jq is already the newest version (1.6-2.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda/envs/310-iris

  added / updated specs:
    - dask
    - google-cloud-aiplatform
    - google-cloud-bigquery
    - ipykernel
    - ipython
    - jupyter
    - notebook
    - numpy
    - openpyxl
    - pandas
    - pandas-gbq
    - pyarrow
    - python=3.10
    - tabulate


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  aiohttp            conda-forge/linux-64::aiohttp-3.8.6-py310h2372a71_1 
  aiosignal          conda-forge/noarch::aiosignal-1.3.1-pyhd8ed1ab_0 
  anyio              conda-forge/noarch::anyio-4.

usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')

EnvironmentLocationNotFound: Not a conda environment: /opt/conda/envs/310-test

config/install_env_kernel.sh: line 30: ipython: command not found
usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'deactivate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')


Updated display_name in /opt/conda/envs/310-iris/share/jupyter/kernels/python3/kernel.json to Python (310-iris)


usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')
usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'deactivate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')


Virtual environment '310-iris' created and added to Jupyter kernels.
To use it, start Jupyter Notebook and select 'Python (310-iris)' kernel.


*** **Now you have to restart this notebook and search the new kernel to activate it!** ***

### Install environment dependencies

In [9]:
%%bash
pip install -U bigframes pyarrow

## Executable Code in Vertex Training as Custom Container

### Define input data

In [1]:
# input-data-definition (DON'T REMOVE THIS COMMENT)
project_id = 'ml-framework-maas'
input_files_queries = ['input_data.sql']
valid_test_rate = [0, 0.2]
output_tables = ["model_test_iris.train_x", "model_test_iris.test_x", "model_test_iris.train_y", "model_test_iris.test_y"]

### Code to ingest input data

In [2]:
# input-data-ingestion (DON'T REMOVE THIS COMMENT)
import bigframes.pandas as bf
from sklearn.model_selection import train_test_split
from typing import List

# Auxiliar functions
def modify_query_file(input_name: str, replacements: dict={}, path='../scripts/queries/'):
    """
    Modifies file in place with a dictionary of string replacements
    """
    with open(path+input_name, 'r') as file :
        filedata = file.read()
    if replacements:
        for key, value in replacements.items():
            filedata = filedata.replace(key, value)
    return filedata

# Main function
def input_data_ingestion(
    project_id: str,
    valid_test_rate: List[float],
    location: str='us-central1',
    secret_path: List[str]=None,
    input_files_queries: List[str]=None,
    input_files_storage_uri: List[str]=None,
    test_mode: bool=False,
    labels: List[str]=None,
):
    bf.options.bigquery.location = "us"  # Dataset is in 'us' not 'us-central1'
    bf.options.bigquery.project = project_id

    input_table_query = modify_query_file(input_files_queries[0], replacements={'@PROJECT_ID': project_id}, path='queries/')
    
    df = bf.read_gbq(query_or_table=input_table_query).to_pandas()

    species_categories = {
        "versicolor": 0,
        "virginica": 1,
        "setosa": 2,
    }
    df["species"] = df["species"].map(species_categories)

    # Assign an index column name
    index_col = "index"
    df.index.name = index_col

    feature_columns = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
    label_columns = df[["species"]]
    train_X, test_X, train_y, test_y = train_test_split(
        feature_columns, label_columns, test_size=valid_test_rate[1]
    )

    print("X_train size: ", train_X.size)
    print("X_test size: ", test_X.size)
    
    return (train_X, test_X, train_y, test_y)


### Code to process input data and generate output data

In [3]:
# process (DON'T REMOVE THIS COMMENT)
from sklearn.preprocessing import StandardScaler
from typing import List
import pandas as pd

# Auxiliar functions
# ...

def feature_generation_from_input_data(
    input_data: tuple,
    project_id: str,
    location: str='us-central1',
    secret_path: List[str]=None,
    test_mode: bool=False,
    labels: List[str]=None
):
    train_X = input_data[0]
    test_X = input_data[1]
    train_y = input_data[2]
    test_y = input_data[3]
    
    # Instantiate transformer
    transformer = StandardScaler()

    # Execute transformer on Vertex (train_X is bigframes.dataframe.DataFrame, X_train is np.array)
    scaled_train_X = transformer.fit_transform(train_X)
    train_X = pd.DataFrame(scaled_train_X, index=train_X.index, columns=train_X.columns)

    # Execute transformer on Vertex (test_X is bigframes.dataframe.DataFrame, X_test is np.array)
    scaled_test_X = transformer.transform(test_X)
    train_X = pd.DataFrame(scaled_test_X, index=test_X.index, columns=test_X.columns)
    
    return (train_X, test_X, train_y, test_y)


### Code to save output data

In [4]:
# output-data-storing (DON'T REMOVE THIS COMMENT)

# Auxiliar functions
# ...

def feature_storing(
    feature_data: tuple,
    project_id: str,
    labels: List[str]=None,
    location: str='us-central1',
    output_tables: List[str]=None,
    output_bucket: List[str]=None,
    secret_path: List[str]=None,
    test_mode: bool=False
):
    train_X = feature_data[0]
    test_X = feature_data[1]
    train_y = feature_data[2]
    test_y = feature_data[3]
        
    train_X = train_X.reset_index(drop=True)
    train_y = train_y.reset_index(drop=True)

    test_X = test_X.reset_index(drop=True)
    test_y = test_y.reset_index(drop=True)

    train_X.to_gbq(destination_table=output_tables[0], project_id=project_id, location=location, if_exists='replace')
    test_X.to_gbq(destination_table=output_tables[1], project_id=project_id, location=location, if_exists='replace')
    train_y.to_gbq(destination_table=output_tables[2], project_id=project_id, location=location, if_exists='replace')
    test_y.to_gbq(destination_table=output_tables[3], project_id=project_id, location=location, if_exists='replace')
    
    return tuple([f'{project_id}.{table}' for table in output_tables])


### Code to run every function in steps

### Standalone execution to test the functions
Use a small dataset to have a sucess execution into this machine. Full dataset will be run in Vertex Training with enough resources.

In [35]:
# input variables
project_id = 'ml-framework-maas'
input_files_queries = ['input_data.sql']
valid_test_rate = [0, 0.2]
output_tables = ["model_test_iris.train_x", "model_test_iris.test_x", "model_test_iris.train_y", "model_test_iris.test_y"]

# Steps
input_data = input_data_ingestion(
    project_id=project_id,
    input_files_queries=input_files_queries,
    valid_test_rate=valid_test_rate
)
    
feature_data = feature_generation_from_input_data(
    project_id=project_id,
    input_data=input_data
)
 
feature_location = feature_storing(
    project_id=project_id,
    feature_data=feature_data,
    output_tables=output_tables
)

print(feature_location)

X_train size:  480
X_test size:  120


100%|██████████| 1/1 [00:00<00:00, 13231.24it/s]

('ml-framework-maas.model_test_iris.train_x', 'ml-framework-maas.model_test_iris.test_x', 'ml-framework-maas.model_test_iris.train_y', 'ml-framework-maas.model_test_iris.test_y')


## Create Vertex Training Job config

### Create the requirements file 

In [53]:
%%bash
chmod +x config/extract_dependencies.sh
bash config/extract_dependencies.sh

### Convert notebook to Python Script

In [32]:
%%bash
python config/convert_notebook2script.py

### Write function tests


Create a dataset for testing your functions in a manner that ensures a smooth transition to production, you should follow a structured approach.  
This includes understanding the nature of your production data, replicating its structure in a testing environment, and ensuring that your test data adequately represents various scenarios and edge cases.

* Define input values to run the functions with the test dataset.
* Include the way to get the test dataset for functions. It must be small no more than 100 samples and very varied data.
* Use this test dataset to run every created function.

#### Define input values to run the functions with the test dataset

In [36]:
import pandas as pd

# input variables
project_id = 'ml-framework-maas'
input_files_queries = ['test_input_data.sql']
valid_test_rate = [0, 0.2]
output_tables = ["model_test_iris.test_mode_train_x", "model_test_iris.test_mode_test_x", "model_test_iris.test_mode_train_y", "model_test_iris.test_mode_test_y"]

In [37]:
def modify_query_file(input_name: str, replacements: dict={}, path='../scripts/queries/'):
    """
    Modifies file in place with a dictionary of string replacements
    """
    with open(path+input_name, 'r') as file :
        filedata = file.read()
    if replacements:
        for key, value in replacements.items():
            filedata = filedata.replace(key, value)
    return filedata

#### Way to get the test dataset for functions

In [38]:
query2create_test_dataset = modify_query_file('test_input_data_generation.sql', replacements={'@PROJECT_ID': project_id}, path='queries/')
df_test_dataset = pd.read_gbq(query=query2create_test_dataset, project_id=project_id)

In [39]:
df_test_dataset

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.5,2.3,1.3,0.3,setosa
1,4.4,3.2,1.3,0.2,setosa
2,4.8,3.0,1.4,0.1,setosa
3,4.8,3.4,1.9,0.2,setosa
4,4.8,3.4,1.6,0.2,setosa
5,7.2,3.6,6.1,2.5,virginica
6,7.4,2.8,6.1,1.9,virginica
7,6.7,3.1,5.6,2.4,virginica
8,4.9,2.5,4.5,1.7,virginica
9,7.7,3.8,6.7,2.2,virginica


#### Run functions with the test dataset

In [40]:
input_data = input_data_ingestion(
    project_id=project_id,
    input_files_queries=input_files_queries,
    valid_test_rate=valid_test_rate
)

X_train size:  48
X_test size:  12


In [41]:
input_data[0]

,sepal_length,sepal_width,petal_length,petal_width
index,,,,
14,7.2,3.6,6.1,2.5
8,4.5,2.3,1.3,0.3
4,4.9,2.5,4.5,1.7
9,7.4,2.8,6.1,1.9
5,6.5,2.8,4.6,1.5
3,4.8,3.4,1.9,0.2
13,6.7,3.1,5.6,2.4
0,5.6,2.9,3.6,1.3
10,7.7,3.8,6.7,2.2


In [42]:
feature_data = feature_generation_from_input_data(
    project_id=project_id,
    input_data=input_data
)

In [43]:
feature_data[0]

,sepal_length,sepal_width,petal_length,petal_width
index,,,,
2,0.490646,-0.186469,0.201966,0.019335
7,-0.115446,-0.186469,0.150400,0.019335
12,-0.894707,0.932343,-1.190311,-1.256757


In [44]:
feature_location = feature_storing(
    project_id=project_id,
    feature_data=feature_data,
    output_tables=output_tables
)

100%|██████████| 1/1 [00:00<00:00, 9198.04it/s]


In [45]:
feature_location

('ml-framework-maas.model_test_iris.test_mode_train_x',
 'ml-framework-maas.model_test_iris.test_mode_test_x',
 'ml-framework-maas.model_test_iris.test_mode_train_y',
 'ml-framework-maas.model_test_iris.test_mode_test_y')

### Define data quality rules

Encourage users to fill out these fields with as much accuracy and detail as possible.  
Accurate metadata can significantly enhance data understanding, cleaning, processing, and analysis.  
It will help in making informed decisions about data handling, modeling, and interpretation of results.  

File that must be filled: **'tests/data_quality_metrics.xlsx'**

Please fill out the following fields for each feature in the dataset. 


1. **column_name**: 
   - **Description:** The name of the column or field in your dataset.
  
2. **column_type**: 
   - **Description:** The data type of the column (e.g., integer, float, string, date).

3. **min_value**: 
   - **Description**: The minimum value in the column. It's the smallest number in the case of numeric data.
   - **Formula**: $\text{min(Column)}$

4. **mean_value**: 
   - **Description**: The average value of the column, calculated by summing all values and then dividing by the count of values.
   - **Formula**: $\text{Mean} = \frac{\sum_{i=1}^{n} X_i}{n}$

5. **max_value**: 
   - **Description**: The maximum value in the column. It's the largest number in the case of numeric data.
   - **Formula**: $\text{max(Column)}$

6. **outliers_count**: 
   - **Description**: The count of values significantly different from others. Outliers are often determined using statistical methods like the Z-score, where $\text{k}$ is a threshold value (commonly 1.96 for 95% confidence).
   - **Formula**: Count values where $|X_i - \text{Mean}| > k \times \text{Standard Deviation}$

7. **nulls_count**: 
   - **Description**: The number of missing or undefined values in the column.
   - **Formula**: Count of `Null` or `NaN` values in the Column.

8. **rows_count**: 
   - **Description**: Simply the count of all rows or records in the dataset.
   - **Formula**: Total number of rows in the dataset.

9. **cardinality**: 
   - **Description**: The number of distinct values in the column.
   - **Formula**: Count of unique values in the Column.

10. **selectivity**: 
    - **Description**: The ratio of unique values to total rows, indicating how many rows can be filtered using a particular value.
    - **Formula**: $\text{Selectivity} = \frac{\text{Number of Unique Values}}{\text{Total Number of Rows}}$

11. **density**: 
    - **Description**: The proportion of non-null values to total values in the column, indicating how "filled" a column is with non-null data.
    - **Formula**: Typically calculated as $\text{Density} = \frac{\text{Non-Null Count}}{\text{Total Count}}$


In [46]:
from IPython.display import display, HTML
import pandas as pd

df = pd.read_excel('tests/data_quality_metrics.xlsx')
html_table = df.to_html()

display(HTML(html_table))

,column_name,column_type,min_value,mean_value,max_value,outliers_count,nulls_count,rows_count,cardinality,selectivity,density


### Create docker container

In [54]:
%%bash
chmod +x config/build-img.sh
bash config/build-img.sh -m iris_classification -p 3.10

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


Sending build context to Docker daemon  402.4kB
Step 1/9 : ARG PYTHON_VERSION=3.9
Step 2/9 : FROM python:${PYTHON_VERSION}-slim-buster
 ---> 93b9055430ce
Step 3/9 : ARG PROJECT_DIRECTORY="/usr/app"
 ---> Using cache
 ---> 64ecca38c846
Step 4/9 : WORKDIR ${PROJECT_DIRECTORY}
 ---> Using cache
 ---> d2946e60194d
Step 5/9 : RUN apt update && apt-get install g++ -yy
 ---> Using cache
 ---> 5ab8176babc3
Step 6/9 : COPY requirements.txt requirements.txt
 ---> Using cache
 ---> b87a06f16002
Step 7/9 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 581e3d3693f9
Step 8/9 : COPY . .
 ---> 0b9146342259
Step 9/9 : ENTRYPOINT ["python3", "src/main.py"]
 ---> Running in b8073fe01531
Removing intermediate container b8073fe01531
 ---> 368b29568773
Successfully built 368b29568773
Successfully tagged us-central1-docker.pkg.dev/ml-framework-maas/datascience-test3/mvp/iris_classification/preprocessing:latest
Deleted Images:
untagged: us-central1-docker.pkg.dev/ml-framework-maas/datascience-te

### Launch Vertex Training Job

In [55]:
%%bash
python config/find_suitable_gcp_machine.py --cpu_cores=5 --ram_gb=8

{'cpu_machine_name': 'n1-standard-8', 'cpu_machine_cores': 8, 'cpu_machine_ram': 30}


In [56]:
%%bash
chmod +x config/launch_custom_job.sh
bash config/launch_custom_job.sh \
-m 'iris_classification' \
-c 'n1-standard-4' \
-i 'us-central1-docker.pkg.dev/ml-framework-maas/datascience-test3/mvp/iris_classification/preprocessing'

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/1099093996594/locations/us-central1/customJobs/1465807123341377536] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/1099093996594/locations/us-central1/customJobs/1465807123341377536

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/1099093996594/locations/us-central1/customJobs/1465807123341377536


------------------------------------------------------------------------------------------------------------------------

## Experimental code

In [ ]:
import bigframes.pandas as bf
from sklearn.model_selection import train_test_split

In [ ]:
bf.options.bigquery.location = "us"  # Dataset is in 'us' not 'us-central1'
bf.options.bigquery.project = 'ml-framework-maas'

In [ ]:
df = bf.read_gbq("bigquery-public-data.ml_datasets.iris").to_pandas()

In [ ]:
species_categories = {
    "versicolor": 0,
    "virginica": 1,
    "setosa": 2,
}
df["species"] = df["species"].map(species_categories)

# Assign an index column name
index_col = "index"
df.index.name = index_col

In [ ]:
feature_columns = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
label_columns = df[["species"]]
train_X, test_X, train_y, test_y = train_test_split(
    feature_columns, label_columns, test_size=0.2
)

In [ ]:
print("X_train size: ", train_X.size)
print("X_test size: ", test_X.size)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Instantiate transformer
transformer = StandardScaler()

# Execute transformer on Vertex (train_X is bigframes.dataframe.DataFrame, X_train is np.array)
X_train = transformer.fit_transform(train_X)
     
# Execute transformer on Vertex (test_X is bigframes.dataframe.DataFrame, X_test is np.array)
X_test = transformer.transform(test_X)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate model, warm_start=True for uptraining
model = LogisticRegression(warm_start=True)

# Train model on Vertex
model.fit(train_X, train_y)

In [ ]:
predictions = model.predict(test_X)

print(f"Remote predictions: {predictions}")

In [ ]:
# User must convert bigframes to pandas dataframe for local evaluation
train_X_pd = train_X.reset_index(drop=True)
train_y_pd = train_y.reset_index(drop=True)

test_X_pd = test_X.reset_index(drop=True)
test_y_pd = test_y.reset_index(drop=True)
    

In [ ]:
# Evaluate model's accuracy score
print(f"Train accuracy: {model.score(train_X_pd, train_y_pd)}")

print(f"Test accuracy: {model.score(test_X_pd, test_y_pd)}")